## SET UP

In [1]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [2]:

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

KeyboardInterrupt: 

In [4]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Set up vector store

In [5]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

In [6]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [7]:
# Save FAISS database to local file
faiss_db_path = "./faiss_db"
vector_store.save_local(faiss_db_path)
print(f"FAISS database saved to {faiss_db_path}")


FAISS database saved to ./faiss_db


## Load Document

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./BankRelationships.pdf"
loader = PyPDFLoader(file_path)

In [4]:
docs = loader.load()

In [5]:
docs[3]

Document(metadata={'producer': 'Acrobat Distiller 6.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2006-04-27T15:44:50+07:00', 'moddate': '2006-04-27T15:44:50+07:00', 'title': 'Microsoft Word - Du_RelationshipVn.doc', 'author': 'sianh', 'source': './BankRelationships.pdf', 'total_pages': 36, 'page': 3, 'page_label': '4'}, page_content='Chương trình giảng dạy kinh tế Fulbright M ối quan hệ Nhà nước – Doanh nghiệp Nhà nước – Ngân hàng \nthương mại Nhà nước ở Việt Nam \n \nHuỳnh Thế Du   4 \n \nCÁC TỪ VIẾT TẮT \n \n \nCAPM: Mô hình định giá tài sản vốn \nCPH: Cổ phần hoá \nDAF: Quỹ Hỗ trợ phát triển \nDNDD: Doanh nghiệp dân doanh \nDNNN hay SOE: Doanh nghiệp nhà nước \nFDIEs: Doanh nghiệp có vốn đầu tư nước ngoài \nHTNHVN: hệ thống ngân hàng Việt Nam \nIMF: Quỹ tiền tệ quốc tế \nKHNN: kế hoạch nhà nước \nNHCT hay ICB: Ngân hàng Công thương Việt Nam \nNHĐT hay BIDV: Ngân hàng Đầu tư và Phát triển Việt Nam \nNHNN hay SBV : Ngân hàng Nhà nước Việt Nam \nNHNNo hay AGR

In [6]:

docs[0]

Document(metadata={'producer': 'Acrobat Distiller 6.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2006-04-27T15:44:50+07:00', 'moddate': '2006-04-27T15:44:50+07:00', 'title': 'Microsoft Word - Du_RelationshipVn.doc', 'author': 'sianh', 'source': './BankRelationships.pdf', 'total_pages': 36, 'page': 0, 'page_label': '1'}, page_content='CHƯƠNG TRÌNH  GIẢNG DẠY KINH TẾ FULBRIGHT \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nMỐI QUAN HỆ NHÀ NƯỚC - DOANH NGHIỆP NHÀ \nNƯỚC - NGÂN HÀNG THƯƠNG MẠI NHÀ NƯỚC  \nỞ VIỆT NAM* \n \n \n \nHuỳnh Thế Du \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nTháng 10 năm 2005 \n \n                                                \n \n* Những nội dung, nhận xét, bình luận trong bài nghiên cứu này chỉ là ý kiến riêng của tác giả mà \nkhông phải là công bố của  Chương trình Giảng dạy Kinh tế Fulbright .')

## Add Documents to Vector Store


In [13]:
# Add documents to vector store
# After adding documents, save again to persist the changes
vector_store.add_documents(docs)
vector_store.save_local("./faiss_db")
print("Documents added and saved to FAISS database")


Documents added and saved to FAISS database


In [ ]:
## Retrieve Documents from FAISS


In [ ]:
# Method 1: Simple similarity search
query = "What is the relationship between banks?"
results = vector_store.similarity_search(query, k=3)
print(f"Found {len(results)} documents")
print("\n--- First Result ---")
print(results[0].page_content[:500])  # Print first 500 characters


In [14]:
# Method 5: Retrieve from saved FAISS database
# First, load the saved database
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
loaded_vector_store = FAISS.load_local(
    "./faiss_db", 
    embeddings, 
    allow_dangerous_deserialization=True
)

# Now retrieve from loaded database
query = "banking partnerships"
results = loaded_vector_store.similarity_search(query, k=2)
print(f"Retrieved {len(results)} documents from saved database")
for i, doc in enumerate(results, 1):
    print(f"\n--- Document {i} ---")
    print(f"Page: {doc.metadata.get('page', 'N/A')}")
    print(doc.page_content[:400] + "...")


Retrieved 2 documents from saved database

--- Document 1 ---
Page: 20
Chương trình giảng dạy kinh tế Fulbright M ối quan hệ Nhà nước – Doanh nghiệp Nhà nước – Ngân hàng 
thương mại Nhà nước ở Việt Nam 
 
Huỳnh Thế Du   21 
(VCB), Ngân hàng Công th ương Việt Nam (ICB) và Ngân hàng Nông nghi ệp và Phát 
triển nông thông (AGRB).  
Với tốc độ tăng trưởng tín d ụng của hệ thống ngân hàng Vi ệt Nam trong nh ững năm 
gần đây được coi là t ương đối nóng (g ấp đôi mức tăng G...

--- Document 2 ---
Page: 20
Chương trình giảng dạy kinh tế Fulbright M ối quan hệ Nhà nước – Doanh nghiệp Nhà nước – Ngân hàng 
thương mại Nhà nước ở Việt Nam 
 
Huỳnh Thế Du   21 
(VCB), Ngân hàng Công th ương Việt Nam (ICB) và Ngân hàng Nông nghi ệp và Phát 
triển nông thông (AGRB).  
Với tốc độ tăng trưởng tín d ụng của hệ thống ngân hàng Vi ệt Nam trong nh ững năm 
gần đây được coi là t ương đối nóng (g ấp đôi mức tăng G...


In [ ]:
# Method 6: Similarity search with metadata filtering
# Example: Search only in specific pages
query = "bank relationships"
results_filtered = vector_store.similarity_search(
    query, 
    k=5,
    filter={"page": 0}  # Only search in page 0
)
print(f"Found {len(results_filtered)} documents in page 0")
